In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dask
import dask.dataframe as dd
import time
import os
from IPython.display import display
import ast
import warnings
warnings.filterwarnings('ignore')
# import sweetviz as sv


# print(dask.__version__)

In [2]:
# Convert JSON to parquet files

# Paths to save the Parquet files
# parquet_dir = "../../data/processed/"

# # 1. 
# # Convert pandas DataFrames to Parquet
# data_tip = pd.read_json(f"../../data/raw/yelp_academic_dataset_tip.json", lines=True)
# data_tip.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_tip.parquet"))

# # Convert Dask DataFrames to Parquet
# data_businesses = dd.read_json(f"../../data/raw/yelp_academic_dataset_business.json", lines=True)
# data_businesses.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_business.parquet"))

# data_checkin = dd.read_json(f"../../data/raw/yelp_academic_dataset_checkin.json", lines=True)
# data_checkin.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_checkin.parquet"))

# data_user = dd.read_json(f"../../data/raw/yelp_academic_dataset_user.json", lines=True)
# data_user.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_user.parquet"))

# # Read the large JSON file with Dask in chunks and convert to Parquet
# data_review = dd.read_json(f"../../data/raw/yelp_academic_dataset_review.json", lines=True, blocksize="64MB")
# data_review.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review.parquet"))



# Read and save sample data for larger files
# data_user = dd.read_json('../../data/raw/yelp_academic_dataset_user.json', lines=True, blocksize=40000000)  # 1MB blocksize
# data_user_sample = data_user.head(5000)  # Adjust the number of rows to read
# data_user_sample.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review_user.parquet"))

# # Review Data
# data_review = dd.read_json(f"../../data/raw/yelp_academic_dataset_review.json", lines=True, blocksize=40000000)
# data_review = data_review.head(5000)
# data_review.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review_sample.parquet"))



In [3]:

# Load data and set indices
data_businesses = dd.read_parquet("../../data/processed/yelp_academic_dataset_business.parquet")
data_businesses = data_businesses.set_index('business_id')

data_checkin = dd.read_parquet("../../data/processed/yelp_academic_dataset_checkin.parquet")
data_checkin = data_checkin.set_index('business_id')

data_tip = dd.read_parquet("../../data/processed/yelp_academic_dataset_tip.parquet")

data_review_sample = dd.read_parquet("../../data/processed/yelp_academic_dataset_review_sample.parquet")
data_review_sample = data_review_sample.set_index('review_id')

data_user_sample = dd.read_parquet("../../data/processed/yelp_academic_dataset_user_sample.parquet")
data_user_sample = data_user_sample.set_index('user_id')


df_dict = {
    'bus_df': data_businesses,
    'checkin_df': data_checkin,
    'tip_df': data_tip,
    'review_df': data_review_sample,
    'user_df': data_user_sample
}


In [4]:

# Define the processing instructions
processes = {
    "bus_df": {
        "bool": ["is_open"],
        "dict": ["attributes", "hours"],
        "lists": ["categories"]
    },
    "checkin_df": {
        "date_time_list": ["date"]
    },
    "tip_df": {
        "date_time": ["date"],
        "string": ["text"]
    },
    "user_df": {
        "date_time": ["yelping_since"], #yelping since already shows in datetime...
        "lists": ["elite", "friends"]
    },
    "review_df": {
        "date_time": ["date"],
        "string": ["text"]
    }
}

for df_name, df in df_dict.items():
    
    instructions = processes[df_name]

    for dtype, cols in instructions.items():
        if dtype == "bool":
            for col in cols:
                df[col] = df[col].astype(bool)
        
        elif dtype == "date_time":
            for col in cols:
                df[col] = dd.to_datetime(df[col], errors='coerce')
                
        elif dtype == "string":
            for col in cols:
                df[col] = df[col].str.lower()
        
        elif dtype == "date_time_list":
            # continue
            for col in cols:
                df[col] = df[col].str.split(', ').map(
                    lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S", errors='coerce') 
                    if isinstance(x, list) else [],
                    meta=('x', 'object')
                )
        elif dtype == "dict":
            # continue
            for col in cols:
                df[col] = df[col].map(
                    lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) else {}, 
                    meta=('x', 'object')
                )
        
        elif dtype == "lists":
            # continue
            for col in cols:
                df[col] = df[col].map(
                    lambda x: x.split(', ') if pd.notna(x) and isinstance(x, str) else [], 
                    meta=('x', 'object')
                )
        
        

    # df_dict[df_name] = df

# If you need to compute the results, you can do so here:
# for df_name in df_dict:
#     df_dict[df_name] = df_dict[df_name].compute()

In [5]:
# Save dataset

# # Assuming df_dict contains Dask DataFrames
# for name, ddf in df_dict.items():
#     ddf.to_parquet(f'../../data/processed/{name}.parquet')  # or engine='fastparquet'


In [6]:
# data_checkin.compute().describe() #taking too long...

for df_name, df in df_dict.items():
    if df_name != "checkin_df":
        print(df_name)
        display(df.compute().describe())
        

bus_df


,latitude,longitude,stars,review_count
count,150346.000000,150346.000000,150346.000000,150346.000000
mean,36.671150,-89.357339,3.596724,44.866561
std,5.872759,14.918502,0.974421,121.120136
min,27.555127,-120.095137,1.000000,5.000000
25%,32.187293,-90.357810,3.000000,8.000000
50%,38.777413,-86.121179,3.500000,15.000000
75%,39.954036,-75.421542,4.500000,37.000000
max,53.679197,-73.200457,5.000000,7568.000000


tip_df


,date,compliment_count
count,908915,908915.000000
mean,2015-06-14 10:13:53.302817280,0.012525
min,2009-04-16 13:11:49,0.000000
25%,2013-01-26 01:18:02.500000,0.000000
50%,2015-03-15 01:10:25,0.000000
75%,2017-08-02 06:19:55.500000,0.000000
max,2022-01-19 20:38:55,6.000000
std,NaN,0.120763


review_df


,stars,useful,funny,cool,date
count,5000.000000,5000.00000,5000.000000,5000.000000,5000
mean,3.838600,0.86880,0.252400,0.340000,2015-04-13 15:23:19.728400128
min,1.000000,0.00000,0.000000,0.000000,2005-03-12 03:47:06
25%,3.000000,0.00000,0.000000,0.000000,2013-10-25 00:11:22
50%,4.000000,0.00000,0.000000,0.000000,2015-09-23 23:04:05.500000
75%,5.000000,1.00000,0.000000,0.000000,2017-03-15 19:22:29
max,5.000000,34.00000,19.000000,13.000000,2018-10-04 18:10:01
std,1.357983,1.71947,0.904244,0.968601,NaN


user_df


,review_count,yelping_since,useful,funny,cool,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
count,4211.000000,4211,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000,4211.000000
mean,341.816671,2009-05-08 13:15:50.810496512,1275.876276,684.314890,883.991926,44.503681,3.830869,93.666588,11.848730,9.772738,8.834244,5.336737,56.616481,144.450724,128.450962,128.450962,48.126098,37.050344
min,1.000000,2005-01-03 18:01:35,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,42.000000,2008-03-15 06:30:46,60.000000,15.000000,19.000000,2.000000,3.600000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,145.000000,2009-06-18 21:29:13,244.000000,79.000000,101.000000,10.000000,3.830000,5.000000,2.000000,1.000000,0.000000,0.000000,7.000000,9.000000,10.000000,10.000000,4.000000,1.000000
75%,399.000000,2010-08-03 00:38:08.500000,868.000000,344.500000,436.500000,36.000000,4.060000,33.000000,7.000000,3.000000,3.000000,2.000000,28.000000,45.000000,51.000000,51.000000,22.500000,6.000000
max,9941.000000,2013-07-30 17:49:45,124311.000000,91842.000000,113069.000000,3138.000000,5.000000,12391.000000,3575.000000,5662.000000,1744.000000,2261.000000,4901.000000,16674.000000,13262.000000,13262.000000,6574.000000,14045.000000
std,582.808313,NaN,4488.603479,2908.094407,3755.493255,142.928365,0.431512,433.074690,69.657934,98.640596,51.187551,44.008681,218.024305,714.616047,562.896788,562.896788,223.647099,342.897864
